
# Diagnostic Models for Precipitation

The following notebook will demonstrate how to use diagnostic models inside of Earth-2
MIP for transforming outputs of global weather models into different quantities of
interest. More information on diagnostics can be found in the [user guide](https://nvidia.github.io/earth2mip/userguide/diagnostic.html).

In summary this notebook will cover the following topics:

- Loading a built in diagnostic model for predicting total precipitation
- Combining the diagnostic model with a prognostic model using the DiangosticLooper


In [ ]:
import datetime
import os

## Loading Diagnostic Models
Loading diagnostic models is similar to prognostic models, but presently use a
slightly different API. In this example we will using the built in FourCastNet AFNO
to serve as the underlying prognostic model that will drive the time-integration. The
`PrecipitationAFNO` model will then be used to "post-process" the outputs of
this model to predict precipitation. The key API to load a diagnostic model is the
`load_diagnostic(package)` function which takes a model package in. If you're
interested in using the built in model package (i.e. checkpoint), then the
`load_package()` function can do this for you.



In [ ]:
from modulus.distributed.manager import DistributedManager
from earth2mip.networks import get_model
from earth2mip.diagnostic import PrecipitationAFNO

device = DistributedManager().device

print("Loading FCN model")
model = get_model("e2mip://fcn", device=device)

print("Loading precipitation model")
package = PrecipitationAFNO.load_package()
diagnostic = PrecipitationAFNO.load_diagnostic(package)

The next step is to wrap the prognostic model with the Diagnostic Time loop.
Essentially, this adds the execution of the diagnostic model on top of the forecast
model iterator. This will add the total precipitation field (`tp`) to the output data
which can the be further processed.



In [ ]:
from earth2mip.diagnostic import DiagnosticTimeLoop

model_diagnostic = DiagnosticTimeLoop(diagnostics=[diagnostic], model=model)

## Running Inference
With the diagnostic time loop created the final steps are to create the data source
and run inference. For this example we will use the CDS data source again, using cached
data when working with default dates. You can also try different dates, however it is assumed
that your CDS API key is already set up. Reference the
[first example](https://nvidia.github.io/earth2mip/examples/01_ensemble_inference.html#set-up)
for additional information. We will use the basic inference workflow which returns a
Xarray dataset we will save to netCDF.

In [ ]:
from earth2mip.inference_ensemble import run_basic_inference
from earth2mip.initial_conditions import cds, gfs, hdf5

print("Constructing initializer data source")
dat_source = 'cds' # 'cds', 'gfs'

if dat_source == 'gfs':
    data_source = gfs.DataSource(model.in_channel_names)
    time = datetime.datetime(2024, 4, 9)
elif dat_source == 'cds':
    data_source = cds.DataSource(model.in_channel_names)
    time = datetime.datetime(2017, 8, 24, 12)
elif dat_source == 'hdf5':
    h5_folder = '/e2ws_data/era5_73var'
    data_source = hdf5.DataSource.from_path(
        root=h5_folder, channel_names=model.in_channel_names
    )
    time = datetime.datetime(2017, 8, 24, 12)

print("Running inference")
output_dir = "outputs/04_diagnostic_precip"
os.makedirs(output_dir, exist_ok=True)
ds = run_basic_inference(
    model_diagnostic,
    n=36,
    data_source=data_source,
    time=time,
)
ds.to_netcdf(os.path.join(output_dir, "precipitation_afno.nc"))
ds

## Post Processing
With inference complete we can do some post processing on our predictions.
Hurricane Harvey was selected for inference due to its intense rain fall over
the Gulf coast of Texas and Louisiana. Lets plot the total precipitation that
occurred over Houston.

In [ ]:
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

location = 'houston' # 'munich', 'san francisco'

if location == 'san francisco':
    loc_lat, loc_lon = 37.75, 237.5 # Lon is [0, 360]
elif location == 'munich':
    loc_lat, loc_lon = 48., 11.5
elif location == 'houston':
    loc_lat, loc_lon = 29.75, 264.75 # 84.75
else:
    raise ValueError(f'region {location} not implemented.')

plt.close("all")
# Open dataset from saved NetCDFs
ds = xr.open_dataarray(os.path.join(output_dir, "precipitation_afno.nc"))
tp = ds.sel(channel="tp", lat=loc_lat, lon=loc_lon)

plt.figure(figsize=(12,5))
plt.plot(pd.to_datetime(tp.coords["time"]), tp.values*1000)
plt.title(f"{location} (lat: {loc_lat}N lon: {loc_lon}E)")
plt.ylabel("Total Precipitation (mm)")
plt.savefig(f"{output_dir}/tp.png")

The intense rain while Harvey was over Houston is clearly visible, lets have a look at the
regional contour of the area to better understand the structure of this event.

In [ ]:
plt.close("all")

# Open dataset from saved NetCDFs
ds = xr.open_dataarray(os.path.join(output_dir, "precipitation_afno.nc"))
nsteps = 4
proj = ccrs.AlbersEqualArea(central_latitude=loc_lat, central_longitude=loc_lon)
fig, ax = plt.subplots(
    1,
    nsteps,
    figsize=(20, 6.5),
    subplot_kw={"projection": proj},
    gridspec_kw={"wspace": 0.05, "hspace": 0.007},
)
if nsteps == 1:
    ax = [ax]

for step in range(nsteps):
    i = step + 3
    tp = ds[i, 0].sel(channel="tp")

    ax[step].add_feature(cartopy.feature.OCEAN, zorder=0)
    ax[step].add_feature(cartopy.feature.LAND, zorder=0)
    ax[step].add_feature(cartopy.feature.COASTLINE,lw=.5)
    ax[step].add_feature(cartopy.feature.RIVERS,lw=.5)
    ax[step].add_feature(cartopy.feature.BORDERS, linewidth=0.6, edgecolor='dimgray')

    masked_data = np.ma.masked_where(tp.values < 0.001, tp.values)
    img = ax[step].pcolormesh(
        ds.lon,
        ds.lat,
        1000 * masked_data,
        transform=ccrs.PlateCarree(),
        cmap="plasma",
        vmin=0,
        vmax=20,
    )
    ax[step].set_title(pd.to_datetime(ds.coords["time"])[i])
    ax[step].coastlines(color="k")
    ax[step].set_extent([loc_lon-10., loc_lon+10., loc_lat-7.5, loc_lat+7.5], ccrs.PlateCarree())
    if step == nsteps-1:
        plt.colorbar(img, ax=ax, shrink=.3, orientation="horizontal",)


ax[0].text(
    -0.07,
    0.55,
    "Total Precipitation (mm)",
    va="bottom",
    ha="center",
    rotation="vertical",
    rotation_mode="anchor",
    transform=ax[0].transAxes,
)
plt.tight_layout
plt.savefig(f"{output_dir}/diagnostic_bay_area_tp.png")

<div class="alert alert-warning"><h4>Exercise</h4><p>

In this notebook you have produced a global precipitation hindcast and
analysed the rain over Houston. Feel free to explore other locations,
San Francisco and Munich are already provided, you can also add further
places which are close to your heart.

Since some of you might have plans for a barbecue this week, let's check
if the weather will be dry! To do so, change the data source to GFS and
select a data in the near past, eg this morning. Can the plans go ahead,
or should you do an indoor activity instead?

</p></div>


This completes the introductory notebook on running diagnostic models. Diagnostic
models are significantly cheaper to train than generative models like CorrDiff. 
Such models have a range of other advantages though, which we will explore in 
the final exercise of this course.
